In [1]:
# !pip install chromedriver-binary==119.0.6045.105

In [46]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from urllib.parse import urlparse, parse_qs
from tqdm import tqdm
import chromedriver_binary
import time
import pandas as pd

In [47]:
race_info_df = pd.read_csv('race_info_df.csv')
race_info_df = race_info_df.drop('Unnamed: 0', axis=1)
race_info_df.head(1)

,date,place_name,grade,when,title,title_url,racelist_url,odds_url,beforeinfo_url,pcexpect_url,myexpect_url,raceresult_url,resultlist_url,replay_url,stadium_url
0,20131101,桐生,is-ippan,is-nighter,第４６回サンケイスポーツ杯,https://www.boatrace.jp/owpc/pc/race/raceindex...,https://www.boatrace.jp/owpc/pc/race/racelist?...,https://www.boatrace.jp/owpc/pc/race/odds3t?rn...,https://www.boatrace.jp/owpc/pc/race/beforeinf...,https://www.boatrace.jp/owpc/pc/race/pcexpect?...,https://www.boatrace.jp/owpc/pc/race/myexpect?...,https://www.boatrace.jp/owpc/pc/race/raceresul...,https://www.boatrace.jp/owpc/pc/race/resultlis...,https://www.boatrace.jp/owpc/pc/data/stadium?j...,javascript:MultiOpen('http://livebb.jlc.ne.jp/...


In [48]:
# オプションを設定してヘッドレスモードを有効にする
chrome_options = Options()
chrome_options.add_argument("--headless")  # GUIをオフに
chrome_options.add_argument("--disable-gpu")  # GPUハードウェアアクセラレーションをオフに。Windows環境では必要な場合があります。
chrome_options.add_argument("--no-sandbox")  # サンドボックスプロセスを無効に。Linux環境の一部のセキュリティ制約を回避するために必要な場合があります。

# ヘッドレスモードでChromeを起動
driver = webdriver.Chrome(options=chrome_options)
# driver = webdriver.Chrome()

In [49]:
for url in list(race_info_df['racelist_url'])[:10]:
    print(url)

https://www.boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=01&hd=20131101
https://www.boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=02&hd=20131101
https://www.boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=04&hd=20131101
https://www.boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=08&hd=20131101
https://www.boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=10&hd=20131101
https://www.boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=12&hd=20131101
https://www.boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=13&hd=20131101
https://www.boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=14&hd=20131101
https://www.boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=16&hd=20131101
https://www.boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=18&hd=20131101


In [ ]:
data_list = []
for url in tqdm(list(race_info_df['racelist_url'])):
    for i in range(1, 13):
        time.sleep(0.2)
#         url = 'https://www.boatrace.jp/owpc/pc/race/racelist?rno=1&jcd=04&hd=20131101'
        target_url = f'https://www.boatrace.jp/owpc/pc/race/racelist?rno={i}&jcd' + url.split('&jcd')[-1]

        # 日付
        date = url[-8:]

        # URLを解析してクエリパラメータを取得
        parsed_url = urlparse(url)
        query_params = parse_qs(parsed_url.query)

        # レース場ID
        place_id = query_params.get('jcd', [None])[0]

        # レース番号
        race_number = i

        # スクレイピング対象のURLにアクセス
        driver.get(target_url)

        table_x_path = '/html/body/main/div/div/div/div[2]/div[5]/table'
        table = driver.find_element(by=By.XPATH, value=table_x_path)
        tbody_elements = table.find_elements(by=By.TAG_NAME, value='tbody')

        for tbody in tbody_elements:
            d_list = []
            d_list.append(date)
            d_list.append(place_id)
            d_list.append(race_number)

            tr_elements = tbody.find_elements(by=By.TAG_NAME, value='tr')
            tr = tr_elements[0]
            td_elements = tr.find_elements(by=By.TAG_NAME, value='td')
            # 枠
            no = td_elements[0].text
            d_list.append(no)
            # 登録番号
            register_number = td_elements[2].find_elements(by=By.TAG_NAME, value='div')[0].text.split('/')[0]
            d_list.append(register_number)
            # クラス
            class_ = td_elements[2].find_elements(by=By.TAG_NAME, value='div')[0].find_element(by=By.TAG_NAME, value='span').text
            d_list.append(class_)
            # 名前
            name = td_elements[2].find_elements(by=By.TAG_NAME, value='div')[1].find_element(by=By.TAG_NAME, value='a').text
            d_list.append(name)
            # 名前URL
            name_url = td_elements[2].find_elements(by=By.TAG_NAME, value='div')[1].find_element(by=By.TAG_NAME, value='a').get_attribute('href')
            d_list.append(name_url)
            # 支部
            texts = td_elements[2].find_elements(by=By.TAG_NAME, value='div')[2].text
            subdivision = texts.split('\n')[0].split('/')[0]
            d_list.append(subdivision)
            #出身地
            place_of_birth = texts.split('\n')[0].split('/')[1]
            d_list.append(place_of_birth)
            # 年齢
            old = texts.split('\n')[1].split('/')[0].replace('歳', '')
            d_list.append(old)
            # 体重
            weight = texts.split('\n')[1].split('/')[1].replace('kg', '')
            d_list.append(weight)

            # F数
            f_num = td_elements[3].text.split('\n')[0]
            d_list.append(f_num)
            # L数
            l_num = td_elements[3].text.split('\n')[1]
            d_list.append(l_num)
            # 平均ST
            st_mean = td_elements[3].text.split('\n')[2]
            d_list.append(st_mean)

            data_list.append(d_list)

  0%|                  | 9/46034 [33:52<2809:38:54, 219.77s/it]

In [ ]:
columns = [
    'date', 'place_id', 'race_number', 'no', 'register_number', 'class', 'name',
    'name_url', 'subdivision', 'place_of_birth', 'old', 'weight', 'f_num', 'l_num', 'st_mean'
]

race_df = pd.DataFrame(data_list, columns=columns)
race_df.head()

In [ ]:
race_df.to_csv('race_df.csv')